In [5]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
script_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

In [3]:
import src.config as config

In [4]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Feature view is defined to read data from feature store, define how you want to fetch data from potentially many different feature groups and merge it to get the final dataset. 
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feaeture view already existed. Skip creation")

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/835736/fs/831559/fv/time_series_hourly_feature_view/version/1


In [7]:
# feature-view-obj.func(): gets training data-frame from hopsworks
ts_data, _ = feature_view.training_data(
    description="Time-series hourly taxi rides"
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (20.79s) 
2024-07-03 11:39:35,578 WARNING: VersionWarning: Incremented version to `1`.



In [8]:
# sort ts-data-frame and print few values to make sure its there
ts_data.sort_values(by=["pickup_location_id", "pickup_hour"], inplace=True)
ts_data

,pickup_hour,num_of_rides,pickup_location_id
3318347,2022-01-01 00:00:00+00:00,0,1
1294678,2022-01-01 01:00:00+00:00,0,1
4280032,2022-01-01 02:00:00+00:00,0,1
1164401,2022-01-01 03:00:00+00:00,0,1
4135778,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
3246573,2024-07-03 11:00:00+00:00,7,265
3250388,2024-07-03 12:00:00+00:00,7,265
3249426,2024-07-03 13:00:00+00:00,5,265
3247600,2024-07-03 14:00:00+00:00,4,265


In [12]:
from src.data import transform_ts_data_into_features_and_target
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="numpy")


# Transform time-series-data into training-data which is features/targets
features, targets = transform_ts_data_into_features_and_target(
    ts_data,              # pass iong ts-dataframe
    input_seq_len=24*28,  # input is rides of previous month
    step_size=23,         # 
) # returns feautres-df 

features_and_target = features.copy()  # combine feautres/targets, add new column target-rides-next-hour equal to targets-df
features_and_target["target_rides_next_hour"] = targets
print(f"{features_and_target.shape=}") # (examples, idk)
# keep clearing this cells output

2024-07-03 11:57:30,003 WARNING: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



 96%|█████████▌| 248/259 [01:08<00:04,  2.54it/s]

2024-07-03 11:57:30,420 WARNING: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



 96%|█████████▌| 249/259 [01:08<00:04,  2.49it/s]

2024-07-03 11:57:30,828 WARNING: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



...

2024-07-03 11:57:34,397 WARNING: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



100%|█████████▉| 258/259 [01:13<00:00,  2.08it/s]

2024-07-03 11:57:34,885 WARNING: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



100%|██████████| 259/259 [01:13<00:00,  3.53it/s]


features_and_target.shape=(217878, 675)


In [23]:
from datetime import date, timedelta, datetime
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# THIS IS MY OWN CUTOFF DATE FOR QUICK FIX
cutoff_date=pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

TypeError: '<' not supported between instances of 'str' and 'Timestamp'